<a href="https://colab.research.google.com/github/LastPeek1/Lab-2/blob/main/%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D0%B8%D0%B7%D0%B2%D0%BB%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D1%8F_%D0%B8%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D0%B8_%D0%B8%D0%B7_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0_%D0%BE%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81%D1%83%D1%89%D0%BD%D0%BE%D1%81%D1%82%D0%B5%D0%B9%2C_%D1%81%D0%BE%D0%B1%D1%8B%D1%82%D0%B8%D0%B9_%D0%B8_%D0%BE%D1%82%D0%BD%D0%BE%D1%88%D0%B5%D0%BD%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu
!pip install spacy
!pip install transformers
!pip install torch
!pip install aiogram

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.7/588.7 kB 11.1 MB/s eta 0:00:00


In [4]:
!pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 12.2 MB/s eta 0:00:00


In [2]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 40.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import logging
import spacy
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
import telebot

# Инициализация логирования
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

# Загрузка модели из HuggingFace
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = TFAutoModel.from_pretrained("cointegrated/rubert-tiny")

# Проверка и настройка использования GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("GPU успешно настроен для использования TensorFlow.")
    except:
        print("Не удалось настроить GPU для TensorFlow.")
else:
    print("GPU не обнаружен. Будет использоваться CPU.")

# Создание экземпляра бота
bot = telebot.TeleBot('6197987721:AAFK9kl6IgStvKUBaOf19ybn2hj6SSIJTHo')

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, 'Привет! Отправьте мне текст, и я извлеку из него информацию.')

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    text = message.text

    # Обработка текста с помощью spaCy
    doc = nlp(text)

    # Извлечение сущностей
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    # Извлечение событий (глаголы)
    events = [token.lemma_ for token in doc if token.pos_ == "VERB"]

    # Извлечение отношений (субъект-глагол-объект)
    relations = []
    for token in doc:
        if token.dep_ == 'ROOT':
            subject = [w.text for w in token.lefts if w.dep_ == 'nsubj']
            object_ = [w.text for w in token.rights if w.dep_ == 'obj']
            if subject and object_:
                relations.append((subject[0], token.lemma_, object_[0]))

    # Получение эмбеддингов с помощью модели ruBERT
    inputs = tokenizer(text, return_tensors="tf")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state  # Размерность [batch_size, sequence_length, hidden_size]

    # Формирование ответа
    response = "🏷 **Найденные сущности:**\n"
    for ent_text, ent_label in entities:
        response += f"- {ent_text} ({ent_label})\n"

    response += "\n⚡️ **Найденные события:**\n"
    for event in events:
        response += f"- {event}\n"

    response += "\n🔗 **Извлеченные отношения:**\n"
    for subj, verb, obj in relations:
        response += f"- {subj} — {verb} — {obj}\n"

    # Отправка ответа пользователю
    bot.reply_to(message, response, parse_mode='Markdown')

if __name__ == '__main__':
    bot.polling()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already u

Не удалось настроить GPU для TensorFlow.


Token indices sequence length is longer than the specified maximum sequence length for this model (879 > 512). Running this sequence through the model will result in indexing errors
